In [8]:
from cellworld import *
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm.notebook import tqdm
import pickle
from random import choices
import matplotlib.colors as colors
import matplotlib.cm as cmx
import pandas as pd

from _src.pose import *
from _src.visibility import *
from _src.itor import *

In [2]:
# load experiment
e = Experiment.load_from_file('./_data/MICE_20220608_1557_DMM3_21_05_SR5_experiment.json')
vis,w = get_vis(e)

# # load visibility matrix
[A,V,pts,sparse_arr] = pickle.load(open('./_data/visibility-21_05-1000.pkl','rb'))   
vis_graph = {'V':V,'A':A,'src':pts,'dst':sparse_arr}

In [11]:
# build a pose library
poselib = pd.DataFrame(columns=['mouse','experiment','episode','frame','prey_pose','pred_loc','pred_angle'])
for j,ep in enumerate(e.episodes):
    pt = ep.trajectories.where('agent_name','prey').get_unique_steps()
    pt = ep.trajectories.where('agent_name','predator')
    for step in pt:
        if step.data:
            # parse the pose
            pose = PoseList.parse(step.data)
            
            # get predator info if available
            rind = np.where(np.array(rt.get('frame'))==step.frame)[0]
            if len(rind) > 0:
                # update location
                rloc = rt[rind[0]].location

,mouse,experiment,episode,frame,prey_pose,pred_loc,pred_angle


In [ ]:
# for each step in the trajectory, compute true ITOR val
pt = e.episodes[episode].trajectories.where('agent_name','prey').get_unique_steps()
body_parts=['body_mid','tail_base','tail_post_base','tail_pre_tip','tail_tip']
#body_parts=['body_mid']
ITOR = []
ITOR_body = []
for i in tqdm(range(len(pt))):
    step = pt[i]
    if step.data:
        pose = PoseList.parse(step.data)

        # compute true ITOR
        I = compute_itor_pose(pose,
                  step.rotation,
                  vis_graph,
                  head_parts=['head_base'],
                  body_parts=body_parts)
        ITOR.append(I['ITOR'])
        
        # compute ITOR with just the body
        I = compute_itor_pose(pose,
                  step.rotation,
                  vis_graph,
                  head_parts=['head_base'],
                  body_parts=['body_mid'])
        ITOR_body.append(I['ITOR'])